## Install

In [ ]:
!pip3 install huggingface-hub

In [ ]:
!pip install --upgrade openai
!pip install --upgrade pydantic

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
# !mkdir MetaMath-Mistral-7B
# !huggingface-cli download meta-math/MetaMath-Mistral-7B --local-dir MetaMath-Mistral-7B --local-dir-use-symlinks False

In [ ]:
# !pip install sentencepiece

In [ ]:
# VERSION = "1.11"
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
# !python pytorch-xla-env-setup.py --version $VERSION  > /dev/null

## Import library

In [ ]:
# import tensorflow as tf
# import tensorflow_hub as hub
# print("Tensorflow version " + tf.__version__)
# AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
# # Detect TPU, return appropriate distribution strategy
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() 

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
# # PyTorch XLA-specific imports
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.debug.metrics as met

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [4]:
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# wandb.login(key = secret_wandb)
# run = wandb.init(
# #     project='Fine tuning MetaMath mistral 7B - ZAIC',
#     project='Fine tuning openchat 7B - ZAIC',
#     job_type="training", 
#     anonymous="allow"
# )

In [6]:
# base_model = "meta-math/MetaMath-Mistral-7B"
# base_model = "gpt2-xl"
base_model = "gpt2"
# base_model = "openchat/openchat_3.5"
new_model = "BK-BigAI-Math"
model_hotamath_path = "/kaggle/working/BK-BigAI-Math"

## Download dataset

In [ ]:
!mkdir dataset

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/test.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)
hf_hub_download(repo_id="hotamago/ZAIC-2023", filename="Elementary Maths Solving/train.zip", revision="main", repo_type="dataset", local_dir="dataset", local_dir_use_symlinks=False)

In [ ]:
!sudo apt-get install unzip

In [ ]:
!mkdir datasetRaw
!unzip -q -o "dataset/Elementary Maths Solving/test.zip" -d "datasetRaw"
!unzip -q -o "dataset/Elementary Maths Solving/train.zip" -d "datasetRaw"

## Load dataset

In [7]:
import os
import json
import re
import time

In [8]:
train_data = None
test_data = None
with open(os.path.join("datasetRaw", "train", "/kaggle/working/datasetRaw/math_train.json"), "r") as f:
    train_data = json.loads(f.read())['data']
with open(os.path.join("datasetRaw", "test", "/kaggle/working/datasetRaw/math_test.json"), "r") as f:
    test_data = json.loads(f.read())['data']

In [9]:
train_data[0]

{'id': '1',
 'question': 'Một người bán hàng bỏ ra 80,000 đồng tiền vốn và bị lỗ 6%. Để tính số tiền lỗ ta phải tính',
 'choices': ['A. 80,000 : 6',
  'B. 80,000 x 6',
  'C. 80,000 : (6 x 100)',
  'D. (80,000 x 6) : 100'],
 'explanation': 'Theo đề bài, số tiền lỗ bằng 6% của 80 000 đồng . Để tìm số tiền lỗ ta có thể lấy 80 000 chia cho 100 rồi nhân với 6 (tức là 80 000 : 100 × 6) hoặc lấy 80000 nhân với 6 rồi chia cho 100 (tức là 80 000 × 6 : 100).',
 'answer': 'D. (80,000 x 6) : 100'}

In [10]:
test_data[0]

{'id': '01-0203',
 'question': 'Một cửa hàng đã bán 30% số hàng hiện có và thu được 15 000 000 đồng. Hỏi nếu bán hết hàng thì cửa hàng thu được bao nhiêu tiền?',
 'choices': ['A. 4 500 000 đồng',
  'B. 45 000 000 đồng',
  'C. 50 000 000 đồng',
  'D. 450 000 000 đồng']}

In [11]:
MAX_TOKEN_MODEL = 512

In [12]:
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "<|endoftext|>" # "<|end_of_turn|>" # "</s>"
DEFAULT_BOS_TOKEN = "<|endoftext|>" # "<s>"
DEFAULT_UNK_TOKEN = "<|endoftext|>" # "<unk>"
DEFAULT_BOI_TOKEN = "<|human|>" # "Human:" # "[INST]"
DEFAULT_EOI_TOKEN = "<|assistant|>" # "Assistant:" # "[/INST]"
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with the choices, one of the choices is the correct answer to the request. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Choices:\n{choices}"
    ),
    "prompt_input_run": (
        DEFAULT_BOI_TOKEN + " Below is an instruction that describes a task. paired with the choices, one of the choices is the correct answer to the request. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Choices:\n{choices}"
        "\n" + DEFAULT_EOI_TOKEN + " \n\n"
#         "### Explanation:\n Let's think step by step.\n"
        "### Explanation:\n Hãy suy nghĩ từng bước một.\n"
    ),
}

In [13]:
timeGlobal = 0
def startTime():
    global timeGlobal
    timeGlobal = time.time()
def getTime():
    return (time.time() - timeGlobal)

In [14]:
def ApplyPromptTemplate(instruction, choices, typeP = "prompt_input"):
    return PROMPT_DICT[typeP].format(instruction = instruction, choices = "\n".join(choices))

### Add explantion by GPT 3.5 Tubo

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key=user_secrets.get_secret("OPENAI_API_KEY"),
#     organization='org-j48waUrvSOM1n0J1SLXiAr8n',
)

def autoGPTAddExplantion(problem, answer):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "Explan by Vietnamese step-by-step for given answer to given problem.\nRule:\n- No markdown format\n- Given answer always true for given problem\n- No title\n- Short explantion\n"},
        {"role": "user", "content": f"### Problem:\n{problem}\n\n### Answer:\n{answer}"},
        {"role": "system", "content": "Giải thích: "},
      ],
#       max_tokens=512,
      temperature=0,
      top_p=1.0,
#       top_k=50,
    )
    return response.choices[0].message.content

In [ ]:
import time

In [ ]:
startTime()
cntSt = 0
i = 0
timeWait = 20
while i < len(train_data):
    singleData = train_data[i]
    if "explanation" in singleData.keys():
        i += 1
        continue
    print(f"Runing testcase ({i})")
    
    try:
        res = autoGPTAddExplantion(singleData['question'], singleData['answer'])
    except Exception as inst:
        print(type(inst))
        print(inst)
        
        time.sleep(timeWait)
        timeWait *= 2
        if timeWait > 80:
            break
        print
        continue
    
    timeWait = 20
        
    cntSt += 1
    train_data[i]['explanation'] = res
    print(f"Done testcase ({i})")
    #     print(train_data[i])
    if cntSt%3 == 0:
        deltaTime = getTime()
        if (60 - deltaTime) > -1:
            time.sleep(60 - deltaTime + 2)
        startTime()
        cntSt = 0
    
    i += 1
print("Done all testcase")

In [ ]:
with open("/kaggle/working/datasetRaw/math_train.json", "w", encoding='utf-8') as f:
    json.dump({
        "__count__": len(train_data),
        "data": train_data
    }, f)

### Add data by education program

Todo: Create data generation algorithms for each chapter.

Below is class base, change \_call function to a function that create radom question for that chapter

In [ ]:
class BaseGenChapter:
    def __init__(*args, **kwargs):
        pass
    
    def _call(self, *args, **kwargs) -> str:
        raise NotImplementedError("_call not Implement yet")
    
    def call(self, *args, **kwargs) -> str:
        return self._call(*args, **kwargs)

### Preprocess data

In [15]:
from datasets import Dataset
import random

In [16]:
# random.shuffle(train_data)

In [17]:
num_train_dataset = len(train_data)
valition_radio = 0.1
tokenized_train_dataset_raw = train_data[:-int(num_train_dataset*valition_radio)]
tokenized_val_dataset_raw = train_data[-int(num_train_dataset*valition_radio):]

In [18]:
datasetStruct = {"input":[], "output":[]}
dataset = {"text":[]}
num_train_dataset = len(tokenized_train_dataset_raw)
for i in range(num_train_dataset):
    ttdro = tokenized_train_dataset_raw[i]
    
    if "explanation" not in ttdro.keys():
        continue
    
    input_content = "{0} {1}".format(
        DEFAULT_BOI_TOKEN,
        ApplyPromptTemplate(ttdro['question'], ttdro['choices']),
    )
    datasetStruct["input"].append(input_content)
    
    if "explanation" not in ttdro.keys():
        output_content = "\n{0} \n\n{1}\n\n{2} {3}".format(
            DEFAULT_EOI_TOKEN,
            "### Explanation:\nNo explanation{0}",
            "### Answer:\n{0}".format(ttdro['answer']),
            DEFAULT_EOS_TOKEN,
        )
    else:
        output_content = "\n{0} \n\n{1}\n\n{2} {3}".format(
            DEFAULT_EOI_TOKEN,
            "### Explanation:\n{0}".format(ttdro['explanation']),
            "### Answer:\n{0}".format(ttdro['answer']),
            DEFAULT_EOS_TOKEN,
        )
    datasetStruct["output"].append(output_content)
    
    dataset["text"].append(input_content + output_content)
#     <s>[INST][/INST] </s>

In [19]:
print(dataset["text"][32])

<|human|> Below is an instruction that describes a task, paired with the choices, one of the choices is the correct answer to the request. Write a response that appropriately completes the request.

### Instruction:
Ngày thứ nhất, bác Thái thu hoạch được 250 kg nhãn. Ngày thứ hai, số ki- lô-gam nhãn bác Thái thu hoạch được đã giảm đi 2 lần so với ngày thứ nhất. Vậy cả hai ngày bác Thái thu hoạch được số ki-lô-gam nhãn là:

### Choices:
A. 500 kg
B. 750 kg
C. 125kg
D. 375 kg
<|assistant|> 

### Explanation:
Ngày thứ hai, bác Thái thu hoạch được số ki-lô-ham nhãn là: 250 : 2 = 125 (kg) 
 Cả hai ngày bác Thái thu hoạch được số ki-lô-gam nhãn là: $250 + 125 = 375$ (kg)
 Đáp số: 375 kg

### Answer:
D. 375 kg <|endoftext|>


In [20]:
print(len(dataset["text"]))

1080


## Train model

### Load model

In [21]:
# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.bfloat16,
#     bnb_4bit_use_double_quant= False,
# )
model = AutoModelForCausalLM.from_pretrained(
    base_model,
#         model_hotamath_path,
#     load_in_4bit=True,
#     load_in_8bit= True,
#     quantization_config=bnb_config,
#     torch_dtype=torch.bfloat16,
#     torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

In [22]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    model_max_length=MAX_TOKEN_MODEL,
    padding_side="right",
    use_fast=False,
)

In [23]:
special_tokens_dict = {
    'additional_special_tokens': [DEFAULT_BOI_TOKEN, DEFAULT_EOI_TOKEN],
    'pad_token': DEFAULT_PAD_TOKEN,
    'bos_token': DEFAULT_BOS_TOKEN,
    'eos_token': DEFAULT_EOS_TOKEN,
    'unk_token': DEFAULT_UNK_TOKEN,
}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [24]:
print(tokenizer.encode("{0} Hello, how are you? \n{1} I'm fine, thank you!{2}".format(
    DEFAULT_BOI_TOKEN,
    DEFAULT_EOI_TOKEN,
    DEFAULT_EOS_TOKEN,
)))

[50258, 18435, 11, 703, 389, 345, 30, 220, 198, 50257, 314, 1101, 3734, 11, 5875, 345, 0, 50256]


### Check max token

In [25]:
max_token_of_dataset = 0
listLongToken = []
for i in range(len(dataset["text"])):
    text = dataset["text"][i]
    token_len = len(tokenizer.encode(text))
    max_token_of_dataset = max(token_len, max_token_of_dataset)
    if token_len > MAX_TOKEN_MODEL:
        print(i, token_len, "\n")
        listLongToken.append(i)
#         print(text)
print(max_token_of_dataset)

Token indices sequence length is longer than the specified maximum sequence length for this model (673 > 512). Running this sequence through the model will result in indexing errors


71 673 

120 594 

239 637 

329 558 

353 566 

361 779 

367 590 

380 592 

388 584 

396 574 

397 796 

399 645 

400 793 

435 671 

440 590 

454 606 

455 531 

458 716 

463 555 

477 542 

483 632 

484 750 

486 667 

487 808 

491 629 

494 769 

529 565 

536 897 

537 563 

539 841 

540 868 

543 527 

546 694 

563 518 

570 828 

578 514 

589 568 

591 903 

592 635 

603 709 

604 664 

614 525 

617 616 

621 515 

622 726 

660 1383 

662 542 

724 729 

742 696 

754 943 

761 1032 

764 680 

766 908 

784 531 

799 549 

802 845 

814 758 

815 571 

823 533 

839 611 

844 582 

845 647 

847 700 

854 844 

862 592 

864 847 

873 577 

902 537 

927 535 

958 602 

961 536 

965 630 

976 667 

997 514 

998 819 

1000 660 

1013 749 

1023 548 

1024 778 

1026 1131 

1027 626 

1028 1068 

1031 519 

1044 955 

1051 814 

1053 613 

1057 574 

1060 561 

1063 586 

1072 582 

1073 685 

1079 528 

1383


### Remove long token

In [26]:
print(len(listLongToken))

92


In [27]:
dataset["text"] = [dataset["text"][i] for i in range(len(dataset["text"])) if i not in listLongToken]

In [28]:
# Check
max_token_of_dataset = 0
for i in range(len(dataset["text"])):
    text = dataset["text"][i]
    token_len = len(tokenizer.encode(text))
    max_token_of_dataset = max(token_len, max_token_of_dataset)
    if token_len > MAX_TOKEN_MODEL:
        print(i, token_len, "\n")
print(max_token_of_dataset)

512


### Nice stuct dataset

In [29]:
dataset = Dataset.from_dict(dataset)

### DataCollator

In [30]:
from trl import DataCollatorForCompletionOnlyLM

In [31]:
instruction_template = DEFAULT_BOI_TOKEN
response_template = DEFAULT_EOI_TOKEN
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

### Setup model for train

In [32]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)


In [33]:
# model = prepare_model_for_kbit_training(model)
# peft_config = LoraConfig(
#     lora_alpha=16,
#     lora_dropout=0.1,
#     r=64,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
# )
# model = get_peft_model(model, peft_config)

In [34]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=25,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=1000,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    save_total_limit = 1,
    warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="constant",
    report_to="none"
#     report_to="wandb"
)

In [ ]:
# import bitsandbytes as bnb
# from torch import nn
# from transformers.trainer_pt_utils import get_parameter_names

# no_decay = ["bias", "LayerNorm.weight"]
# optimizer_grouped_parameters = [{
#     "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#     "weight_decay": training_arguments.weight_decay,
# },
# {
#     "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#     "weight_decay": 0.0,
# }]


# optimizer_kwargs = {
#     "betas": (training_arguments.adam_beta1, training_arguments.adam_beta2),
#     "eps": training_arguments.adam_epsilon,
# }
# optimizer_kwargs["lr"] = training_arguments.learning_rate
# adam_bnb_optim = bnb.optim.Adam8bit(
#     optimizer_grouped_parameters,
#     betas=(training_arguments.adam_beta1, training_arguments.adam_beta2),
#     eps=training_arguments.adam_epsilon,
#     lr=training_arguments.learning_rate,
# )

In [ ]:
# model = model.to("cuda:0")

In [ ]:
# print(model)

In [35]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [36]:
print_trainable_parameters(model)

trainable params: 124442112 || all params: 124442112 || trainable%: 100.0


In [37]:
trainer = SFTTrainer(
    model=model,
#     model = base_model,
    train_dataset=dataset,
#     peft_config=peft_config,
    max_seq_length=MAX_TOKEN_MODEL,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
    data_collator=collator,
#     optimizers=(adam_bnb_optim, None)
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
model.config.use_cache = False

### Train and save

In [39]:
trainer.train()

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss
1010,0.006200
1020,0.005100
1030,0.004500
1040,0.004900
1050,0.005200
1060,0.003700
1070,0.004700
1080,0.004000
1090,0.004100
1100,0.004100


TrainOutput(global_step=1550, training_loss=0.0010797570225211882, metrics={'train_runtime': 1088.7784, 'train_samples_per_second': 22.686, 'train_steps_per_second': 1.424, 'total_flos': 5776981065216000.0, 'train_loss': 0.0010797570225211882, 'epoch': 25.0})

In [40]:
# model.save_pretrained(new_model)
trainer.model.save_pretrained(new_model)
model.config.to_json_file(os.path.join(new_model, "config.json"))
tokenizer.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True

In [41]:
!zip -r BK-BigAI-Math.zip BK-BigAI-Math

updating: BK-BigAI-Math/ (stored 0%)
updating: BK-BigAI-Math/special_tokens_map.json (deflated 80%)
updating: BK-BigAI-Math/vocab.json (deflated 68%)
updating: BK-BigAI-Math/added_tokens.json (deflated 26%)
updating: BK-BigAI-Math/model.safetensors (deflated 7%)
updating: BK-BigAI-Math/generation_config.json (deflated 24%)
updating: BK-BigAI-Math/merges.txt (deflated 53%)
updating: BK-BigAI-Math/config.json (deflated 52%)
updating: BK-BigAI-Math/tokenizer_config.json (deflated 73%)


In [42]:
from IPython.display import FileLink
FileLink(r'BK-BigAI-Math.zip')

/kaggle/working/BK-BigAI-Math.zip

## Evalution

### Download model

In [ ]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="hotamago/ZAIC-2023-Model", filename="Hota-Math.zip", repo_type="model", local_dir="/kaggle/working/", local_dir_use_symlinks=False)

In [ ]:
!unzip -q -o Hota-Math.zip -d ./

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

### Load model

In [ ]:
dataset = Dataset.from_dict(dataset)

In [ ]:
model_name_or_path = "/kaggle/working/BK-BigAI-Math"
# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.bfloat16,
#     bnb_4bit_use_double_quant= False,
# )
# model = AutoModelForCausalLM.from_pretrained(
#         model_name_or_path,
#         load_in_4bit=True,
#         quantization_config=bnb_config,
#         torch_dtype=torch.bfloat16,
#         device_map="auto",
#         trust_remote_code=True,
# )

# model = AutoModelForCausalLM.from_pretrained(
#         model_name_or_path,
#         torch_dtype=torch.bfloat16,
#         device_map="auto",
#         trust_remote_code=True,
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
#     torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
#     load_in_4bit=True,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [ ]:
# special_tokens_dict = {'additional_special_tokens': ['[INST]','[/INST]']}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

### Setup pipeline with auto answer get

In [55]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.01,
    top_p=0.01,
    top_k=3,
    repetition_penalty=1.1,
    pad_token_id=tokenizer.eos_token_id
)

In [56]:
import random
# globalRegxCompire = "0-9a-zA-Z\.\:\-\^\! "
def niceValueToCompire(x):
#     x = re.sub("[^{0}]".format(globalRegxCompire), "", x)
    x = re.sub("[ \t\n]", "", x)
    return x
def autoLLMFormat(question, choises = None, debug=False):
    prompt_template = ApplyPromptTemplate(question, choises, "prompt_input_run")
    res = pipe(prompt_template)[0]['generated_text']
    if debug:
        print(res)
    x = re.findall("### Answer:[\n ](.+)", res)
    
    if choises == None:
        return x
    
    choises_compare = [niceValueToCompire(choise_pred) for choise_pred in choises]

    if len(x) == 0:
        return choises[random.randrange(0, len(choises))]
    
    x = niceValueToCompire(x[0])
    
    if (x not in choises_compare):
        return choises[random.randrange(0, len(choises))]
    
    for i in range(len(choises_compare)):
        if x == choises_compare[i]:
            return choises[i]
    
    return "wtf"

### Run evalution

In [57]:
count_proc_testcase = 0
count_pass_testcase = 0

In [58]:
while count_proc_testcase < len(tokenized_val_dataset_raw):
    tvdo = tokenized_val_dataset_raw[count_proc_testcase]
    token_len = len(tokenizer.encode(ApplyPromptTemplate(tvdo['question'], tvdo['choices'], "prompt_input_run")))
    if token_len >= 512:
        print("Too much token", token_len)
    
    startTime()
    answer = autoLLMFormat(tvdo['question'], tvdo['choices'], True)
    deltaTime = getTime()

    if answer == tvdo['answer']:
        count_pass_testcase += 1
    
    count_proc_testcase += 1
    print("Testcase {0}, time: {1}, answer: {2} | {3}, Passed: {4}, IsPass: {5}".format(
        count_proc_testcase,
        deltaTime,
        answer,
        tvdo['answer'],
        count_pass_testcase,
        (answer == tvdo['answer'])
    ))

<|human|> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Một cái bánh pizza có giá 64 nghìn đồng. \frac{1}{8} cái bánh có giá bao nhiêu nghìn đồng?

### Choices:
A. 7 nghìn đồng
B. 9 nghìn đồng 
C. 8 nghìn đồng
D. 10 nghìn đồng
<|assistant|> 

### Explanation:
 Hãy suy nghĩ từng bước một.
 Vì vậy, số tiếp theo trong dây số lít quay phần thập phân của kém số cho hơn chục (vì 6 + 3 = 5).
 Tuy nhiên, ép chia đều vào các không làm việc nghìn, ghép yêu cầu tìm cùng một của ki-lông thị rằng chrom·eblink là 100. Do đó, đáp án C. 8 nghìn đồng.

### Answer:
C. 8 nghìn đồng 
Testcase 1, time: 3.9238297939300537, answer: C. 8 nghìn đồng | C. 8 nghìn đồng, Passed: 1, IsPass: True
<|human|> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Một chiếc ca nô xuôi dòng từ A đến B với vận tốc 25 km/giờ. Nếu vận tốc dòng nước là 2,5 k

KeyboardInterrupt: 

## Run public test dataset

In [ ]:
result_test = []
if os.path.exists(os.path.join("result", "result.txt")):
    with open(os.path.join("result", "result.txt"), "r") as f:
        result_test = f.read().split("\n")
count_id = len(result_test)

In [ ]:
!mkdir result

In [ ]:
print(len(test_data))

In [ ]:
while count_id < len(test_data):
    one_test_data = test_data[count_id]
    startTime()
    answer = autoLLMFormat(one_test_data['question'], one_test_data['choices'], False)
    deltaTime = getTime()
    result_test.append("{0}".format(answer))
    count_id += 1
    if count_id%10 == 0:
        with open(os.path.join("result", "result.txt"), "w", encoding='utf-8') as f:
            f.write("\n".join(result_test))
    print("Testcase {0}, time: {1}, answer: {2}".format(count_id, deltaTime, answer))

In [ ]:
with open(os.path.join("result", "result.txt"), "w", encoding='utf-8') as f:
    f.write("\n".join(result_test))

In [ ]:
print("\n".join(result_test))